In [1]:
import pickle
import pandas as pd
import numpy as np
import itertools
import matplotlib.pyplot as plt
import glob as glob
import os
import sys; sys.path.append("../../..")  # Allows access to all the scripts/modules in the larger directory
from utils import calc_loadings
from collections import defaultdict
from sklearn.cross_decomposition import CCA


### Load consolidated dimreduc dataframe

In [2]:
path = '/home/marcush/Data/TsaoLabData/neural_control_output/degraded_decoding_med_batch/degraded_decoding_med_batch_glom.pickle'
with open(path, 'rb') as f:
    dat = pickle.load(f) 
df_dimreduc = pd.DataFrame(dat)

In [32]:
df_dimreduc.keys()

Index(['dim', 'fold_idx', 'train_idxs', 'test_idxs', 'dimreduc_method',
       'dimreduc_args', 'coef', 'score', 'predictions', 'loss',
       'decoding_object', 'Xtrain', 'Xtest', 'Ytrain', 'Ytest', 'decoder',
       'decoder_args', 'thresholds', 'data_file', 'loader', 'loader_args',
       'task_args', 'data_path', 'results_file'],
      dtype='object')

### Load the processed data from both regions

In [37]:
def make_hashable(d):
    return tuple(sorted((key, make_hashable(value)) if isinstance(value, dict) else (key, value)
                        for key, value in d.items()))


unique_hashes = set(make_hashable(d) for d in df_dimreduc['loader_args'])
unique_dicts = [dict(u) for u in unique_hashes]

for u in unique_dicts:
    u['data_path'] = df_dimreduc['data_path'][0] + "/" + df_dimreduc['data_file'][0]
    u['spike_threshold'] = None
    u['trial_threshold'] = None

In [39]:
preload_dict_path = df_dimreduc['data_path'][0] + "/preloaded/preloadDict.pickle"

with open(preload_dict_path, 'rb') as file:
    preloadDict = pickle.load(file)


for arg_dict in unique_dicts:
    arg_tuple = tuple(sorted(arg_dict.items()))


    for args in preloadDict.keys():

        if args == arg_tuple:

            preloadID = preloadDict[arg_tuple]
            loaded_data_path = os.path.dirname(preload_dict_path) + f"/preloaded_data_{preloadID}.pickle"
            
            if arg_dict['region'] == 'AM':
                with open(loaded_data_path, 'rb') as file:
                    AM_loaded_data = pickle.load(file)

            elif arg_dict['region'] == 'ML':
                with open(loaded_data_path, 'rb') as file:
                    ML_loaded_data = pickle.load(file)


### Sum spike rate per trial

In [40]:
AM_spikes = np.sum(AM_loaded_data['spike_rates'], 1)
ML_spikes = np.sum(ML_loaded_data['spike_rates'], 1)

### Fit CCA

In [48]:
manual_CCA_dim = np.inf

cca_dims = np.min((manual_CCA_dim, AM_spikes.shape[-1], ML_spikes.shape[-1])).astype(int)

ccamodel = CCA(n_components=cca_dims)
ccamodel.fit(ML_spikes, AM_spikes)

cca_save_path = os.path.dirname(preload_dict_path) + f"/CCA_{cca_dims}_dims.pickle"

with open(cca_save_path, 'wb') as file:
    pickle.dump(ccamodel, file)

/home/marcush/Data/anaconda3/envs/ncontrol/lib/python3.9/site-packages/sklearn/cross_decomposition/_pls.py:113: ConvergenceWarning: Maximum number of iterations reached
  warnings.warn("Maximum number of iterations reached", ConvergenceWarning)
